In [2]:
import os
from infoclus import InfoClus


RELATIVE_DATA_PATH = os.path.join(os.path.pardir, 'data')
DATA_SET_NAME = 'german_socio_eco'

infoclus = InfoClus(DATA_SET_NAME, RELATIVE_DATA_PATH)

adata file is saved at ..\data\german_socio_eco\german_socio_eco.h5ad
initialization done


In [7]:
infoclus.model.linkage

'single'

In [1]:
import os
os.path.pardir

'..'

In [ ]:
import os

from adata_utils import generate_adata

RELATIVE_DATA_PATH = os.path.join(os.path.pardir, 'data')       # relative path to the data folder 
# data (folder) - dataset (folder) - dataset.csv
DATA_SET_NAME = 'german_socio_eco'

generate_adata(RELATIVE_DATA_PATH, DATA_SET_NAME)  # generate the AnnData object


In [ ]:
from sklearn.cluster import AgglomerativeClustering
import anndata as ad
import os

RELATIVE_DATA_PATH = os.path.join(os.path.pardir, 'data')
DATA_SET_NAME = 'german_socio_eco'

adata = ad.read_h5ad(os.path.join(RELATIVE_DATA_PATH, DATA_SET_NAME, f'{DATA_SET_NAME}.h5ad'))
embedding = adata.obsm['tSNE_1']

model = AgglomerativeClustering(linkage='single', distance_threshold=0, n_clusters=None)
model.fit(embedding)

print(model.labels_)